# Evaluation


In [ ]:
library('caret')

# simulation

In [ ]:
N=1000

In [ ]:
x = runif(N,0,2*pi)
f = function(x)(x-pi)^2/10+sin(x)
y = f(x)
e = rnorm(N,0,1/10)
y = y + e

options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 100)
plot(x,y)

In [ ]:
df = data.frame(x=x,y=y)

In [ ]:
flds = createFolds(1:nrow(df),k=2)
flds

# test/train split

In [ ]:
test_df = df[flds[[1]],]
train_df = df[flds[[2]],]

In [ ]:
dim(test_df)
dim(train_df)

In [ ]:
# build on the testing data
knn_mod = knnreg(y~.,data=train_df,k=5)

In [ ]:
train_preds = predict(knn_mod,train_df)

In [ ]:
RMSE_train = sqrt(mean((train_df$y-train_preds)^2))

In [ ]:
RMSE_train

In [ ]:
test_preds = predict(knn_mod,test_df)

In [ ]:
RMSE_test = sqrt(mean((test_df$y-test_preds)^2))

In [ ]:
RMSE_test

In [ ]:
plot(x,y)
xe = data.frame(x=sort(runif(1000,0,2*pi)))
lines(xe$x,f(xe$x),col='blue',lwd=5)
lines(xe$x,predict(knn_mod,xe),col='red',lwd=5)

notice how the training RMSE is typically lower than the testing RMSE

# k fold cross validation

In [ ]:
flds = createFolds(1:nrow(df),k=5)
flds

In [ ]:
lengths(flds)

In [ ]:
i = 1
test_df = df[flds[[i]],]
train_df = df[unlist(flds[-i]),]

In [ ]:
dim(test_df)
dim(train_df)

In [ ]:
knn_mod = knnreg(y~.,data=train_df,k=10)

train_preds = predict(knn_mod,train_df)
RMSE_train = sqrt(mean((train_df$y-train_preds)^2))

test_preds = predict(knn_mod,test_df)
RMSE_test = sqrt(mean((test_df$y-test_preds)^2))

RMSE_train
RMSE_test

let's put this in a function

In [ ]:
tt_split_eval = function(train_idx,test_idx){
    test_df = df[test_idx,]
    train_df = df[train_idx,]
    
    knn_mod = knnreg(y~.,data=train_df,k=5)

    train_preds = predict(knn_mod,train_df)
    RMSE_train = sqrt(mean((train_df$y-train_preds)^2))

    test_preds = predict(knn_mod,test_df)
    RMSE_test = sqrt(mean((test_df$y-test_preds)^2))

    return(data.frame(train=RMSE_train,
        test=RMSE_test
                ))
}

In [ ]:
vec = c()
for(i in 1:10)
    vec[i] = i^2

vec

In [ ]:
sapply(1:10,function(i)i^2)

In [ ]:
lapply(1:10,function(i)i^2)

In [ ]:
flds = createFolds(1:nrow(df),k=10)
rmses = lapply(1:length(flds),function(i){
    tdf = tt_split_eval(train_idx = unlist(flds[-i]),test_idx = flds[[i]])
    tdf$i = i 
    return(tdf)
})

In [ ]:
rmses[[1]]

In [ ]:
rmses[[2]]

In [ ]:
RMSE = Reduce('rbind',rmses)

In [ ]:
RMSE

In [ ]:
library('reshape2')

In [ ]:
mRMSE = reshape2::melt(RMSE,id.vars='i')
mRMSE

In [ ]:
library('ggplot2')

In [ ]:
ggplot(data=mRMSE,mapping=aes(x=i,y=value,color=variable))+
    geom_point()

in total summary we can summarize the RMSEs

In [ ]:
median(RMSE$test)

In [ ]:
sd(RMSE$test)

# test/train/validate

How can we use this to choose a value of $k$ for KNN? Use a train/validate/test 3-way split

In [ ]:
flds = createFolds(1:nrow(df),k=10)
flds

In [ ]:
i = 1
test_idx = flds[[i]]
trainval_idx = unlist(flds[-i])

In [ ]:
test_df = df[test_idx,]
trainval_df = df[trainval_idx,]

In [ ]:
dim(test_df)
dim(trainval_df)

In [ ]:
tv_flds = createFolds(1:nrow(trainval_df),k=10)
tv_flds

In [ ]:
j=1
val_idx = tv_flds[[j]]
train_idx = unlist(tv_flds[-j])

In [ ]:
train_df = trainval_df[train_idx,]
val_df = trainval_df[val_idx,]

In [ ]:
dim(train_df)
dim(val_df)

In [ ]:
tt_split_eval_k = function(train_idx,val_idx,k=1){
    train_df = trainval_df[train_idx,]
    val_df = trainval_df[val_idx,]
    
    knn_mod = knnreg(y~.,data=train_df,k=k)

    train_preds = predict(knn_mod,train_df)
    RMSE_train = sqrt(mean((train_df$y-train_preds)^2))

    val_preds = predict(knn_mod,val_df)
    RMSE_val = sqrt(mean((val_df$y-val_preds)^2))

    return(data.frame(train=RMSE_train,
        val=RMSE_val
                ))
}

In [ ]:
tt_split_eval_k(train_idx,val_idx,k=5)

In [ ]:
tt_split_eval_k(train_idx,val_idx,k=10)

In [ ]:
RMSE = lapply(1:75,function(k){
    tdf = tt_split_eval_k(train_idx,val_idx,k=k)
    tdf$k = k
    return(tdf)
})
RMSE = Reduce('rbind',RMSE)
head(RMSE)

In [ ]:
mRMSE = melt(RMSE,id.vars='k')
head(mRMSE)

In [ ]:
ggplot(data=mRMSE,mapping=aes(x=k,y=value,color=variable))+
    geom_point()+
    scale_x_sqrt()

In [ ]:
which.min(RMSE$val)

In [ ]:
min_df = RMSE[which.min(RMSE$val),]
min_df

In [ ]:
knn_mod = knnreg(y~.,data=trainval_df,k=min_df$k)

In [ ]:
test_preds = predict(knn_mod,test_df)
RMSE_val = sqrt(mean((test_df$y-test_preds)^2))
RMSE_val

# nested x-validation

can I do this in a cross validated way? yes use nested cross validation!

In [ ]:
# outer loop = split into test and trainval datasets
# inner loop = MBP, split into train/val and search over k

In [ ]:
flds = createFolds(1:nrow(df),k=10)

In [ ]:
TEST_RMSE = rep(NA,length(flds))

for(i in 1:length(flds)){
    
    # split testing from trainval
    test_idx = flds[[i]]
    trainval_idx = unlist(flds[-i])
    test_df = df[test_idx,]
    trainval_df = df[trainval_idx,]
     
    #MODEL BUILDING PROCESS
    tv_flds = createFolds(1:nrow(trainval_df),k=10)
    
    K_seq = seq(1,75)
    VAL_MTX = array(NA,c(length(tv_flds),length(K_seq)))

    # normally wouldn't include these two lines
    TRAIN_MTX = array(NA,c(length(tv_flds),length(K_seq)))
    TEST_MTX = array(NA,c(length(tv_flds),length(K_seq)))
    
    for(j in 1:length(tv_flds)){
        
        val_idx = tv_flds[[j]]
        train_idx = unlist(tv_flds[-j])
        train_df = trainval_df[train_idx,]
        val_df = trainval_df[val_idx,]

        for(k in K_seq){
            knn_mod = knnreg(y~.,data=train_df,k=k)
            val_preds = predict(knn_mod,val_df)
            VAL_MTX[j,k] = sqrt(mean((val_df$y-val_preds)^2))
            
            # normally wouldn't include these lines
            train_preds = predict(knn_mod,train_df)
            test_preds = predict(knn_mod,test_df)
            TRAIN_MTX[j,k] = sqrt(mean((train_df$y-train_preds)^2))
            TEST_MTX[j,k] = sqrt(mean((test_df$y-test_preds)^2))
        }
    }
    
    VAL_K = apply(VAL_MTX,2,mean)
    K_hat = K_seq[which.min(VAL_K)]
    
    knn_mod = knnreg(y~.,data=trainval_df,k=K_hat)
    
    # eval on testing data
    test_preds = predict(knn_mod,test_df)
    TEST_RMSE[i] = sqrt(mean((test_df$y-test_preds)^2))
}

Consider the output of a single run of the **outer** loop:

In [ ]:
ggplot(data=melt(TRAIN_MTX),mapping=aes(x=Var2,y=value,color=as.factor(Var1)))+geom_point()

In [ ]:
ggplot(data=melt(VAL_MTX),mapping=aes(x=Var2,y=value,color=as.factor(Var1)))+geom_point()

In [ ]:
ggplot(data=melt(TEST_MTX),mapping=aes(x=Var2,y=value,color=as.factor(Var1)))+geom_point()

to viz easier, let's average over inner x-validation

In [ ]:
library('tidyr')

In [ ]:
ttv = data.frame(train=apply(TRAIN_MTX,2,mean),
           test=apply(TEST_MTX,2,mean),
           val=apply(VAL_MTX,2,mean),
           k=K_seq)
ttv = ttv%>%pivot_longer(cols=c(train,test,val))

head(ttv)
ggplot(data=ttv,mapping=aes(x=k,y=value,color=name))+geom_point()

In [ ]:
plot(VAL_K)

In [ ]:
K_hat = K_seq[which.min(VAL_K)]
K_hat

In [ ]:
# overall

In [ ]:
TEST_RMSE

In [ ]:
mean(TEST_RMSE)

In [ ]:
plot(VAL_K)
abline(h=mean(TEST_RMSE),col='red')

What was the point of this nested x-validation, really? Its *really* `TEST_RMSE`

In [ ]:
TEST_RMSE

In [ ]:
mean(TEST_RMSE)

How do we build the final model for prediction? Basically pull out the inner loop but now use **all** of the data! (Don't hold out a test set in any simple or fancy way)

In [ ]:
tv_flds = createFolds(1:nrow(df),k=10) # use all df
    
K_seq = seq(1,75)
VAL_MTX = array(NA,c(length(tv_flds),length(K_seq)))

for(j in 1:length(tv_flds)){

    val_idx = tv_flds[[j]]
    train_idx = unlist(tv_flds[-j])
    train_df = df[train_idx,]
    val_df = df[val_idx,]

    for(k in K_seq){
        knn_mod = knnreg(y~.,data=train_df,k=k)
        val_preds = predict(knn_mod,val_df)
        VAL_MTX[j,k] = sqrt(mean((val_df$y-val_preds)^2))
    }
}

VAL_K = apply(VAL_MTX,2,mean)
K_hat = K_seq[which.min(VAL_K)]

In [ ]:
plot(VAL_K)

In [ ]:
K_hat

In [ ]:
# fit with all the data
knn_mod = knnreg(y~.,data=df,k=K_hat)

Boss: Ok, cool, how good will this do? `TEST_RMSE`

In [ ]:
plot(x,y)
lines(xe$x,f(xe$x),col='blue',lwd=5)
lines(xe$x,predict(knn_mod,xe),col='red',lwd=5)